# Explore and cluster the neighborhoods in Toronto.

In [58]:
import pandas as pd
import numpy as np
#!conda install -c conda-forge folium=0.5.0 --yes
import folium
from sklearn.cluster import KMeans
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors


In [9]:
df = pd.read_csv('toronto_hoods.csv')
df_coords = pd.read_csv("https://cocl.us/Geospatial_data")
df_coords.columns = ['Postcode', 'Latitude', 'Longitude']
df_coords = df.merge(df_coords, on=('Postcode'))
df_coords.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


### Visualizing the Postal Codes on the map

In [56]:
latitude = 43.7
longitude = -79.3
map_toronto = folium.Map(location=[Latitude, Longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_coords['Latitude'], df_coords['Longitude'], df_coords['Borough'], df_coords['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 


map_toronto

Foursquare Credientials

In [17]:
# @hidden cell
CLIENT_ID = '--- REDACTED ---' # your Foursquare ID
CLIENT_SECRET = '--- REDACTED ---' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


Functions needed to retrieve and categorize venues

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
def getNearbyVenues(postcodes, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for postcode, lat, lng in zip(postcodes, latitudes, longitudes):
        print(postcode)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            postcode, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Get all the near by venues for all of the postal codes

In [29]:
toronto_venues = getNearbyVenues(postcodes=df_coords['Postcode'],
                                   latitudes=df_coords['Latitude'],
                                   longitudes=df_coords['Longitude']
                                  )

M3A
M4A
M5A
M6A
M7A
M9A
M1B
M3B
M4B
M5B
M6B
M9B
M1C
M3C
M4C
M5C
M6C
M9C
M1E
M4E
M5E
M6E
M1G
M4G
M5G
M6G
M1H
M2H
M3H
M4H
M5H
M6H
M1J
M2J
M3J
M4J
M5J
M6J
M1K
M2K
M3K
M4K
M5K
M6K
M1L
M2L
M3L
M4L
M5L
M6L
M9L
M1M
M2M
M3M
M4M
M5M
M6M
M9M
M1N
M2N
M3N
M4N
M5N
M6N
M9N
M1P
M2P
M4P
M5P
M6P
M9P
M1R
M2R
M4R
M5R
M6R
M7R
M9R
M1S
M4S
M5S
M6S
M1T
M4T
M5T
M1V
M4V
M5V
M8V
M9V
M1W
M4W
M5W
M8W
M9W
M1X
M4X
M5X
M8X
M4Y
M7Y
M8Y
M8Z


### Exploring the data

In [34]:
# View the shape and the first 5 of the dataframe
print(toronto_venues.shape)
toronto_venues.head()


(2255, 7)


,Postcode,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,M3A,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [36]:
# How many unique categories from all the venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 275 uniques categories.


## Analyze Each Postal Code

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add Postcode column back to dataframe
toronto_onehot['Postcode'] = toronto_venues['Postcode'] 

# move Postcode column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [39]:
toronto_onehot.shape

(2255, 276)

Group by postal code and take the mean of the frequency of each category

In [41]:
toronto_grouped = toronto_onehot.groupby('Postcode').mean().reset_index()
toronto_grouped

,Postcode,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,M1C,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,M1E,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
3,M1G,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
4,M1H,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
5,M1J,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
6,M1K,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
7,M1L,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
8,M1M,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.333333,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
9,M1N,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


#### Print each postal code's top 5 most common venues

In [42]:
num_top_venues = 5

for hood in toronto_grouped['Postcode']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Postcode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B----
                  venue  freq
0  Fast Food Restaurant   1.0
1     Accessories Store   0.0
2    Miscellaneous Shop   0.0
3         Movie Theater   0.0
4                 Motel   0.0


----M1C----
                 venue  freq
0                  Bar   1.0
1    Accessories Store   0.0
2   Miscellaneous Shop   0.0
3                Motel   0.0
4  Monument / Landmark   0.0


----M1E----
                venue  freq
0         Pizza Place  0.14
1      Breakfast Spot  0.14
2      Medical Center  0.14
3  Mexican Restaurant  0.14
4        Intersection  0.14


----M1G----
               venue  freq
0        Coffee Shop  0.67
1  Korean Restaurant  0.33
2  Accessories Store  0.00
3  Mobile Phone Shop  0.00
4      Movie Theater  0.00


----M1H----
                  venue  freq
0   Fried Chicken Joint  0.14
1      Hakka Restaurant  0.14
2  Caribbean Restaurant  0.14
3       Thai Restaurant  0.14
4    Athletics & Sports  0.14


----M1J----
                        venue  freq
0                 

#### Put it into a pandas dataframe

In [44]:
# Function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [94]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postcode_venues_sorted = pd.DataFrame(columns=columns)
postcode_venues_sorted['Postcode'] = toronto_grouped['Postcode']

for ind in np.arange(toronto_grouped.shape[0]):
    postcode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

postcode_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,College Stadium
1,M1C,Bar,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Diner
2,M1E,Intersection,Breakfast Spot,Pizza Place,Rental Car Location,Electronics Store,Medical Center,Mexican Restaurant,Donut Shop,Discount Store,Dive Bar
3,M1G,Coffee Shop,Korean Restaurant,Yoga Studio,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
4,M1H,Caribbean Restaurant,Bakery,Hakka Restaurant,Athletics & Sports,Bank,Thai Restaurant,Fried Chicken Joint,Donut Shop,Dog Run,Doner Restaurant


## Cluster the Postal Codes

In [95]:
# set number of clusters
kclusters = 2

toronto_grouped_clustering = toronto_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

 #### create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [96]:
# add clustering labels
postcode_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_coords

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(postcode_venues_sorted.set_index('Postcode'), on='Postcode')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Food & Drink Shop,Park,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Diner
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,French Restaurant,Pizza Place,Hockey Arena,Coffee Shop,Portuguese Restaurant,Yoga Studio,Donut Shop,Diner,Discount Store,Dive Bar
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,1.0,Coffee Shop,Pub,Bakery,Park,Mexican Restaurant,Café,Theater,Gym / Fitness Center,Breakfast Spot,Restaurant
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Event Space,Shoe Store,Miscellaneous Shop,Arts & Crafts Store,Gift Shop,Boutique
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,1.0,Coffee Shop,Gym,Diner,Park,College Auditorium,Seafood Restaurant,Sandwich Place,Burger Joint,Burrito Place,Café


In [97]:
toronto_merged.dropna(subset=['Cluster Labels'], inplace=True)
#toronto_merged[(toronto_merged['Cluster Labels'] IS NAN)]
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype('int')
#toronto_merged

visualize the resulting clusters

In [98]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postcode'], toronto_merged['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

#### Cluster 1 = Call it the "Active Locations" Cluster

In [99]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Food & Drink Shop,Park,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Diner
21,York,0,Park,Pharmacy,Fast Food Restaurant,Market,Women's Store,Colombian Restaurant,Comfort Food Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
35,East York,0,Park,Pizza Place,Coffee Shop,Convenience Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Event Space,Eastern European Restaurant,Dessert Shop
40,North York,0,Park,Airport,Electronics Store,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
49,North York,0,Park,Basketball Court,Bakery,Construction & Landscaping,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
61,Central Toronto,0,Park,Gym / Fitness Center,Swim School,Bus Line,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore
64,York,0,Park,Convenience Store,Yoga Studio,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
66,North York,0,Park,Bank,Convenience Store,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
68,Central Toronto,0,Trail,Park,Sushi Restaurant,Jewelry Store,Yoga Studio,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
77,Etobicoke,0,Park,Pizza Place,Mobile Phone Shop,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore


#### Cluster 2 = Call it the 

In [100]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1,French Restaurant,Pizza Place,Hockey Arena,Coffee Shop,Portuguese Restaurant,Yoga Studio,Donut Shop,Diner,Discount Store,Dive Bar
2,Downtown Toronto,1,Coffee Shop,Pub,Bakery,Park,Mexican Restaurant,Café,Theater,Gym / Fitness Center,Breakfast Spot,Restaurant
3,North York,1,Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Event Space,Shoe Store,Miscellaneous Shop,Arts & Crafts Store,Gift Shop,Boutique
4,Queen's Park,1,Coffee Shop,Gym,Diner,Park,College Auditorium,Seafood Restaurant,Sandwich Place,Burger Joint,Burrito Place,Café
6,Scarborough,1,Fast Food Restaurant,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,College Stadium
7,North York,1,Caribbean Restaurant,Japanese Restaurant,Gym / Fitness Center,Café,Baseball Field,Yoga Studio,Drugstore,Dog Run,Doner Restaurant,Donut Shop
8,East York,1,Fast Food Restaurant,Pizza Place,Intersection,Athletics & Sports,Gastropub,Bank,Pharmacy,Gym / Fitness Center,Ethiopian Restaurant,Empanada Restaurant
9,Downtown Toronto,1,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Fast Food Restaurant,Ramen Restaurant,Middle Eastern Restaurant,Restaurant,Bookstore,Japanese Restaurant
10,North York,1,Bakery,Japanese Restaurant,Pub,Metro Station,Yoga Studio,Drugstore,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
11,Etobicoke,1,Bank,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Diner
